


# Exercise - Build AI investor
We've gained some insights into Publicis Groupe and Omnicom Group through analyzing their annual reports. Now, we're ready to delve deeper into a fundamental analysis of these two companies and explore ways to automate trade submissions based on our findings.

## Exercise Overview and purpose
([Refer to this example for using langchain agents](example/langchain_agent_example.ipynb))
 
This exercise is designed to understand how langchain agents work. It creates an AI-based agent to submit trade based on fundamental analysis. The agent should perform following tasks for a given ```ticker``` : 

[Refer to these functions](utilities/stock_utils.py)
   - Get historic stock prices 
   - Get the financial statement
   - Submit a buy or sell order based on this analysis. 
  






### Task 1. Create tool for fetching last 5 days Stock Data for a given ticker 
Create a tool to retrieve historical stock data from Yahoo Finance. This tool will provide the agent access to price trends, volumes, and other relevant financial metrics.

> 10 mins

In [ ]:
ticker = 'PUB.PA'

from utilities2.stock_utils import get_historical_stock_data
print(get_historical_stock_data(ticker))


### Task 2. Create tool for fetching financial statement from yahoo finance for a given ticker

> 10 mins

In [ ]:
from utilities2.stock_utils import get_detail_statement
print(get_detail_statement(ticker))

### Task 3. Create mock functions to submit buy or sell order for a given ticker 

> 10 mins


### Task 4. Consolidate all the tools created above into a Toolkit
Create toolkit using tools. This toolkit will form the backbone of the agent's data-gathering capabilities.

> 15 mins


In [ ]:
from langchain.chat_models import ChatOpenAI
from utilities2.stock_utils import (
    buy_order,
    get_historical_stock_data,
    get_detail_statement,
    sell_order,
)
from langchain.agents import Tool, initialize_agent


llm_model = ChatOpenAI(model_name="gpt-4", temperature=0)

def stock_tool_list():
    # tool list
    tools = [
        Tool(
            name="GetStockData",
            func=get_historical_stock_data,
            description="Use when you are asked to evaluate or analyze a stock. This will output historic share price data. You should input the the stock ticker to it ",
        ),
        Tool(
            name="GetFinancialStatements",
            func=get_detail_statement,
            description="Use this to get financial statement of the company. With the help of this data companys historic performance can be evaluaated. You should input stock ticker to it",
        ),
        Tool(
            name="SubmitBuyOrder",
            func=buy_order,
            description="Use this to submit a buy order based for the company which is performing good, based on the fundamental analysis. With the help of this a buy order can be submitted on stock exchange. You should input stock ticker to it",
        ),
        Tool(
            name="SubmitSellOrder",
            func=sell_order,
            description="Use this to submit a sell order based for the company which is under performing, based on the fundamental analysis. With the help of this a sell order can be submitted on stock exchange. You should input stock ticker to it",
        ),
    ]

    return tools

### Task 5. Initialize agent with the tool kit created above

```
zero_shot_agent = initialize_agent(
        llm=get_openai_gpt3_16k_chat_model(),
        agent="zero-shot-react-description",
        tools=tools,
        verbose=False,
        max_iteration=4,
        return_intermediate_steps=True,
        handle_parsing_errors=True,
    )
```

> 5

In [ ]:

def get_agent(tools):
    zero_shot_agent = initialize_agent(
        llm=llm_model,
        agent="zero-shot-react-description",
        tools=tools,
        verbose=True,
        max_iteration=4,
        #return_intermediate_steps=True,
        handle_parsing_errors=True,
    )

    print(zero_shot_agent.agent.llm_chain.prompt.template)

    return zero_shot_agent


agent = get_agent(stock_tool_list());


### Task 6. Create custom Agent Prompt which uses the defined tools to perform fundamental analysis and submit trade based on the final analysis

Design a prompt to guide the agent's analysis and define clear evaluation steps. This will ensure that the agent's analysis is structured, comprehensive, and aligned with the investor's decision-making process.

> 10 mins

In [ ]:


def stock_agent_prompt_template():
    prompt = """As a financial advisor, your role is to offer stock recommendations upon receiving a ticker symbol and a request for analysis or fundamental examination. 
    Your responses should be as comprehensive as possible, leveraging the tools available to you:

    GetStockData: Deploy this tool for assessing or analyzing a stock by inputting the stock ticker. This action retrieves historical share price data, which is crucial for the initial phase of your analysis.

    GetFinancialStatements: Utilize this tool to obtain the financial statements of the company. These documents are instrumental in evaluating the company's historical performance. Input the stock ticker to access this data.

    SubmitBuyOrder: If your fundamental analysis indicates that the company's prospects are positive, use this tool to place a buy order. Input the stock ticker for the company you've decided is a worthwhile investment.

    SubmitSellOrder: Conversely, if your fundamental analysis suggests that the company is underperforming, you should use this tool to issue a sell order. Input the stock ticker for the company you believe should be sold off.

    Steps for Analysis:

    Step 1: Begin with the GetStockData tool to compile information on the stock. Output: Stock data.

    Step 2: Acquire the company's historical financial data by using GetFinancialStatements tool. Output: Financial statement.

    Step 3: Based on the company's performance and your investment analysis from the previous steps, decide whether to use SubmitBuyOrder tool or SubmitSellOrder tool. Justify your decision with reasons. Output: Confirmation of whether a Sell order or Buy order has been placed.

    Note: If you encounter any issues with a step, proceed to the next one. Your goal is to use these tools methodically to provide informed stock recommendations.

    Use the following format:

    Question: the input question you must answer
    Thought: you should always think about what to do, Also try to follow steps mentioned above
    Action: the action to take, should be one of [GetStockData, GetFinancialStatements, SubmitBuyOrder, SubmitSellOrder]
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times)
    Thought: I now know the final answer
    Final Answer: the final answer to the original input question
    Begin!

    Question: {input}
    Thought:{agent_scratchpad}"""

    return prompt



### Task 7. Assign the prompt created above to agent

> 1 min

In [ ]:
agent.agent.llm_chain.prompt.template = stock_agent_prompt_template()
print(agent.agent.llm_chain.prompt.template)


### Task 8. Use agent to perform fundamental analysis and submit trade based on the ticker provided. 

> 10 mins

In [ ]:
#agent("Submit a buy or sell order based on your fundamental analysis of PUB.PA")
agent("Perform fundamental analysis of PUB.PA and OMC. Compare the fundamental analysis of these two organization and submit a buy or sell order accordingly")
